# Resnet_7
서현이 v0.01에서 모델만 내걸로 바꾼거  
근데 학습한 폴드가 다를 수 있다고 함.
기존 코드에서 weight decay 0.05(아니 이거 왤케 컸었지..) -> 1e-4,  
cosine annealing eta_min = 1e-6 추가함  
또 img size 224로 줄이고 batch size도 32로 줄임
label smoothing도 0.1에서 0.05로 줄임

# Import

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import timm

from sklearn.model_selection import KFold

# GPU 사용 가능 여부 확인
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'사용 디바이스: {device}')
if torch.cuda.is_available():
    print(f'GPU: {torch.cuda.get_device_name(0)}')

In [ ]:
import random
import numpy as np
import torch

# ============================================================
# 재현성 보장 (Reproducibility)
# ============================================================

SEED = 42

def set_seed(seed=SEED):
    """모든 난수 생성기의 시드를 고정하여 재현성 보장"""
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)  # multi-GPU
    
    # CUDA 결정론적 동작 활성화
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    
    # PyTorch 결정론적 알고리즘 사용 (PyTorch 1.8+)
    torch.use_deterministic_algorithms(True, warn_only=True)
    
    print(f"✅ Random Seed 고정 완료: {seed}")
    print(f"   - Python random: {seed}")
    print(f"   - NumPy: {seed}")
    print(f"   - PyTorch: {seed}")
    print(f"   - CUDA: {seed}")
    print(f"   - cuDNN Deterministic: True")
    print(f"   - cuDNN Benchmark: False")
    print(f"   - PyTorch Deterministic Algorithms: True")

# Seed 적용
set_seed(SEED)

In [ ]:
# import torch
# import multiprocessing

# # Windows 환경에서 num_workers > 0 사용을 위한 필수 설정
# try:
#     # 'spawn' 방식을 명시적으로 설정하여 안전한 프로세스 생성을 유도
#     torch.multiprocessing.set_start_method('spawn', force=True) 
#     print("PyTorch multiprocessing start method set to 'spawn'.")
# except RuntimeError as e:
#     # 이미 설정된 경우를 대비한 예외 처리
#     print(f"Start method already set or error: {e}")

# Data Load

In [ ]:
# 하이퍼파라미터
img_size = 224
batch_size = 32
num_classes = 10
num_epochs = 50
# learning_rate = 0.0003 -> train section
num_workers = 0
version = "7"

# 데이터 경로
base_dir = r'c:\Users\USER\PycharmProjects\DeepLearning-Term-Proj'
driver_csv_path = os.path.join(base_dir, 'data', 'driver_imgs_list.csv')
train_dir = os.path.join(base_dir, 'data', 'imgs', 'train')
test_dir = os.path.join(base_dir, 'data', 'imgs', 'test')
name = f'resnet_{version}'

print(f"Train directory: {train_dir}")
print(f"Test directory: {test_dir}")
print(f"이미지 크기: {img_size}x{img_size}")
print(f"배치 크기: {batch_size}")

In [ ]:
driver_df = pd.read_csv(driver_csv_path)

print(f"고유 운전자 수: {driver_df['subject'].nunique()}명")
print(f"운전자 목록: {sorted(driver_df['subject'].unique())}")

# driver_counts = driver_df['subject'].value_counts().sort_index()

# plt.figure(figsize=(12, 6))
# plt.bar(driver_counts.index.astype(str), driver_counts.values, color='C0', alpha=0.9)
# plt.xlabel('Driver')
# plt.ylabel('Image Count')
# plt.title('Images per Driver')
# plt.xticks(rotation=45, ha='right')
# plt.grid(axis='y', alpha=0.3)
# plt.tight_layout()
# plt.show()

# 5-Fold Cross Validation
fold별 운전자 목록을 미리 나누긴 하되 초기 실험 단계에서는 한 폴드(fold 2, 학습 데이터가 가장 많아서..)만 사용하고 이후 마무리 단계에서 전체 폴드 다 돌려서 일반화 성능 끌어올리는 방향으로 진행.

## split train data into 5 folds

In [ ]:
all_drivers = sorted(driver_df['subject'].unique())

n_folds = 5
kfold = KFold(n_splits=n_folds, shuffle=True, random_state=42)

fold_splits = []
for fold_idx, (train_indices, val_indices) in enumerate(kfold.split(all_drivers)):
	train_drivers = [all_drivers[i] for i in train_indices]
	val_drivers = [all_drivers[i] for i in val_indices]

	fold_splits.append({
		'fold': fold_idx+1,
		'train_drivers': train_drivers,
		'val_drivers': val_drivers
	})

	print("Fold", fold_idx+1)
	print("train:", train_drivers, "val:", val_drivers)
	
	train_imgs = driver_df[driver_df['subject'].isin(train_drivers)]
	val_imgs = driver_df[driver_df['subject'].isin(val_drivers)]
	print(f"학습 이미지: {len(train_imgs)}개")
	print(f"검증 이미지: {len(val_imgs)}개")

## define DraiverDataset

In [ ]:
class DriverDataset(Dataset):
    """운전자 행동 데이터셋"""
    
    def __init__(self, data_dir, driver_df, driver_list, transform=None, is_test=False):
        self.data_dir = data_dir
        self.transform = transform
        self.is_test = is_test
        self.images = []
        self.labels = []
        
        if is_test:
            test_images_dir = data_dir
            for img_name in os.listdir(test_images_dir):
                self.images.append(os.path.join(test_images_dir, img_name))
        else: #is_train
            driver_subset = driver_df[driver_df['subject'].isin(driver_list)]

            for _, row in driver_subset.iterrows():
                class_name = row['classname']
                img_name = row['img']
                img_path = os.path.join(data_dir, class_name, img_name)

                self.images.append(img_path)
                class_idx = int(class_name[1:])
                self.labels.append(class_idx)
        print(f"{'테스트' if is_test else '운전자' + str(len(driver_list))+'명'}, 데이터 {len(self.images)}개 이미지")
    
    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, idx):
        img_path = self.images[idx]
        image = Image.open(img_path).convert('RGB')
        
        if self.transform:
            image = self.transform(image)
        
        if self.is_test:
            return image, os.path.basename(img_path)
        else:
            label = self.labels[idx]
            return image, label

## define online transform
서현님 온라인 증강 추가. clahe 적용한 걸로 돌리기에 color jitter만 제외함.

In [ ]:
# transform_train = transforms.Compose([
#     transforms.RandomResizedCrop(img_size, scale=(0.85, 1.0)),
#     transforms.RandomRotation(15),
#     transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.05),
#     transforms.ToTensor(),
#     transforms.Normalize([0.485, 0.456, 0.406],
#                          [0.229, 0.224, 0.225])
# ])

transform_train = transforms.Compose([
    transforms.RandomResizedCrop(224, scale=(0.85, 1.0)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ColorJitter(0.2, 0.2, 0.2, 0.1),
    transforms.ToTensor(),
    transforms.RandomErasing(p=0.3, scale=(0.02, 0.10), ratio=(0.3, 3.3)),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])


transform_eval = transforms.Compose([
    transforms.Resize((img_size, img_size)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])


## define model

### define custom resnet

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import SequentialLR, LinearLR, CosineAnnealingLR
from torch.utils.data import DataLoader
import timm
import numpy as np
from sklearn.metrics import f1_score, log_loss
from tqdm import tqdm

EMBED_DIM = 512
DROPOUT = 0.4
DROP_PATH = 0.1
EARLY_STOP_PATIENCE = 10 

class ResNetDWithHead(nn.Module):
    def __init__(self, num_classes, embed_dim=512, dropout=0.4):
        super().__init__()
        self.backbone = timm.create_model(
            'resnet50d',
            pretrained=True,
            num_classes=0,
            global_pool='',
            drop_path_rate=DROP_PATH,
        )

        self.pool = nn.AdaptiveAvgPool2d((1, 1))

        in_ch = self.backbone.num_features

        self.embed = nn.Sequential(
            nn.Conv2d(in_ch, in_ch, kernel_size=1, bias=False),
            nn.BatchNorm2d(in_ch),
            nn.ReLU(inplace=True),
        )
        self.proj = nn.Sequential(
            nn.Flatten(1),
            nn.Linear(in_ch, embed_dim),
            nn.BatchNorm1d(embed_dim),
            nn.ReLU(inplace=True),
            nn.Dropout(dropout),
        )
        self.classifier = nn.Linear(embed_dim, num_classes)

    def forward_features_map(self, x):
        return self.backbone.forward_features(x)

    def forward(self, x, labels=None):
        x = self.forward_features_map(x)
        x = self.embed(x)
        x = self.pool(x).flatten(1)
        x = self.proj(x)

        logits = self.classifier(x)
        return logits


## train

In [ ]:
def worker_init_fn(worker_id):
    """DataLoader worker의 재현성 보장"""
    worker_seed = torch.initial_seed() % 2**32
    np.random.seed(worker_seed)
    random.seed(worker_seed)

In [ ]:
def train_fold(fold_idx, train_drivers, val_drivers):
    print(f"==== Fold {fold_idx}/{n_folds} ====")

    # ===== 데이터셋 & 로더 =====
    g = torch.Generator()
    g.manual_seed(SEED)

    train_dataset = DriverDataset(
        train_dir, driver_df, train_drivers,
        transform=transform_train, is_test=False
    )
    val_dataset = DriverDataset(
        train_dir, driver_df, val_drivers,
        transform=transform_eval, is_test=False
    )
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True,
                              num_workers=num_workers, pin_memory=True,
                              worker_init_fn=worker_init_fn, generator=g)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False,
                            num_workers=num_workers, pin_memory=True,
                            worker_init_fn=worker_init_fn, generator=g)

    print(f"학습 배치 수: {len(train_loader)}")
    print(f"검증 배치 수: {len(val_loader)}")

    # ===== 모델 =====
    model = ResNetDWithHead(
        num_classes=num_classes,
        embed_dim=EMBED_DIM,
        dropout=DROPOUT,
    ).to(device)

    # # ---- layer1까지 동결 (stem~layer1 freeze) ----
    # for n, p in model.backbone.named_parameters():
    #     if n.startswith('layer2') or n.startswith('layer3') or n.startswith('layer4'):
    #         p.requires_grad = True
    #     else:
    #         p.requires_grad = False
    # # head는 학습
    # for p in model.embed.parameters(): p.requires_grad = True
    # for p in model.proj.parameters(): p.requires_grad = True
    # for p in model.classifier.parameters(): p.requires_grad = True

    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    total_params = sum(p.numel() for p in model.parameters())
    print("\n" + "="*70)
    print("📊 모델 파라미터 통계")
    print("="*70)
    print(f"  전체 파라미터:      {total_params:>15,}")
    print(f"  학습 가능 파라미터:  {trainable_params:>15,} ({100*trainable_params/total_params:>6.2f}%)")
    print("="*70 + "\n")

    # ===== 손실/옵티마/스케줄러 =====
    criterion = nn.CrossEntropyLoss(label_smoothing=0.05)

    # optimizer = optim.AdamW(
    #     filter(lambda p: p.requires_grad, model.parameters()),
    #     lr=3e-4,
    #     weight_decay=5e-2
    # )
    optimizer = optim.AdamW(
        model.parameters(),
        lr=3e-4,
        weight_decay=1e-4
    )
    warmup_epochs = 5
    cosine_epochs = max(1, num_epochs - warmup_epochs)
    scheduler = SequentialLR(
        optimizer,
        schedulers=[
            LinearLR(optimizer, start_factor=0.01, end_factor=1.0, total_iters=warmup_epochs),
            CosineAnnealingLR(optimizer, T_max=cosine_epochs, eta_min=1e-6)
        ],
        milestones=[warmup_epochs]
    )

    # ===== 기록 =====
    history = {'train_loss': [], 'train_acc': [], 'val_loss': [], 'val_acc': [],
               'val_macro_f1': [], 'val_logloss': [], 'learning_rates': []}

    # ===== Early Stopping =====
    early_stop_patience = EARLY_STOP_PATIENCE
    patience_counter = 0
    best_metric = float('inf')
    best_model_path = f'models/{name}.pth'

    print(f"\n⏱️ Early Stopping Patience (val logloss 기준): {early_stop_patience} ep\n")
    print("="*70); print("🚀 학습 시작"); print("="*70)

    def eval_on_loader(model, loader, criterion):
        model.eval()
        total, correct, running_loss = 0, 0, 0.0
        all_probs, all_labels = [], []
        with torch.no_grad():
            pbar = tqdm(loader, desc='Validating', leave=False)
            for images, labels in pbar:
                images = images.to(device, non_blocking=True)
                labels = labels.to(device, non_blocking=True)
                logits = model(images)

                loss = criterion(logits, labels)
                running_loss += loss.item() * labels.size(0)
                preds = logits.argmax(dim=1)
                correct += (preds == labels).sum().item()
                total += labels.size(0)

                probs = logits.softmax(dim=1).detach().cpu().numpy()
                all_probs.append(probs)
                all_labels.append(labels.detach().cpu().numpy())
                pbar.set_postfix({'loss': f'{loss.item():.4f}',
                                  'acc': f'{100.0*correct/max(1,total):.2f}%'})

        avg_loss = running_loss / max(1, total)
        acc = 100.0 * correct / max(1, total)
        all_probs = np.concatenate(all_probs, axis=0)
        all_labels = np.concatenate(all_labels, axis=0)
        macro_f1 = f1_score(all_labels, np.argmax(all_probs, axis=1), average='macro')
        mlogloss = log_loss(all_labels, all_probs, labels=list(range(num_classes)))
        return avg_loss, acc, macro_f1, mlogloss

    # ===== 에폭 루프 =====
    for epoch in range(num_epochs):
        print(f'\n{"="*70}')
        print(f'Epoch {epoch+1}/{num_epochs}')
        print(f'{"="*70}')
        current_lr = optimizer.param_groups[0]['lr']
        history['learning_rates'].append(current_lr)

        # ---- Train ----
        model.train()
        train_loss, train_correct, train_total = 0.0, 0, 0
        pbar = tqdm(train_loader, desc='Training', leave=False)
        for images, labels in pbar:
            images = images.to(device, non_blocking=True)
            labels = labels.to(device, non_blocking=True)
            optimizer.zero_grad(set_to_none=True)
            logits = model(images)

            loss = criterion(logits, labels)
            loss.backward()
            optimizer.step()

            train_loss += loss.item() * labels.size(0)
            preds = logits.argmax(dim=1)
            train_correct += (preds == labels).sum().item()
            train_total += labels.size(0)

            pbar.set_postfix({'loss': f'{loss.item():.4f}',
                              'acc': f'{100.0*train_correct/max(1,train_total):.2f}%',
                              'lr': f'{current_lr:.6f}'})

        epoch_train_loss = train_loss / max(1, train_total)
        epoch_train_acc = 100.0 * train_correct / max(1, train_total)

        # ---- Validate ----
        val_loss, val_acc, val_macro_f1, val_logloss = eval_on_loader(model, val_loader, criterion)
        scheduler.step()

        history['train_loss'].append(epoch_train_loss)
        history['train_acc'].append(epoch_train_acc)
        history['val_loss'].append(val_loss)
        history['val_acc'].append(val_acc)
        history['val_macro_f1'].append(val_macro_f1)
        history['val_logloss'].append(val_logloss)

        print(f'\n📊 Epoch {epoch+1} 결과:')
        print(f'  Train Loss: {epoch_train_loss:.4f} | Train Acc: {epoch_train_acc:.2f}%')
        print(f'  Val   Loss: {val_loss:.4f} | Val Acc:   {val_acc:.2f}%')
        print(f'  Val Macro-F1: {val_macro_f1:.4f} | Val LogLoss: {val_logloss:.4f}')
        print(f'  LR: {current_lr:.6f}')

        # ---- Best / Early Stop ----
        if val_logloss < best_metric:
            best_metric = val_logloss
            patience_counter = 0
            torch.save({
                'fold': fold_idx,
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'history': history,
                'val_loss': val_loss,
                'val_acc': val_acc,
                'val_macro_f1': val_macro_f1,
                'val_logloss': val_logloss,
                'model_name': name,
                'freeze_mode': 'freeze_to_layer1',
                'drop_path_rate': DROP_PATH,
                'trainable_params': trainable_params,
                'total_params': total_params,
                'seed': SEED
            }, best_model_path)
            print(f'  ✅ 최고 성능(val_logloss) 모델 저장! (val_logloss: {val_logloss:.4f})')
        else:
            patience_counter += 1
            print(f'  ⏳ Early Stopping 카운터: {patience_counter}/{EARLY_STOP_PATIENCE}')
            if patience_counter >= EARLY_STOP_PATIENCE:
                print(f'\n{"="*70}')
                print(f'🛑 Early Stopping 발동! (Epoch {epoch+1}) — val_logloss 개선 없음')
                print(f'   최고 val_logloss: {best_metric:.4f}')
                print(f'   모델 저장 경로: {best_model_path}')
                print(f'{"="*70}')
                break

    final_epoch = epoch + 1
    print("\n" + "="*70)
    print(f"✅ Fold {fold_idx} 학습 완료! (EXP-1+)")
    print("="*70)
    print(f"  총 학습 에폭: {final_epoch}/{num_epochs}")
    print(f"  최고 Macro-F1: {max(history['val_macro_f1']):.4f}")
    print(f"  최저 Val Loss: {min(history['val_loss']):.4f}")
    print(f"  최저 LogLoss:  {min(history['val_logloss']):.4f}")
    print(f"  최고 Val Acc:  {max(history['val_acc']):.2f}%")
    print(f"  모델 저장: {best_model_path}")
    print(f"  학습 파라미터: {trainable_params:,} / {total_params:,} ({100*trainable_params/total_params:.2f}%)")
    print("="*70)

    return {
        'fold': fold_idx,
        'history': history,
        'best_macro_f1': max(history['val_macro_f1']),
        'best_val_loss': min(history['val_loss']),
        'best_val_logloss': min(history['val_logloss']),
        'best_val_acc': max(history['val_acc']),
        'model_path': best_model_path,
        'stopped_epoch': final_epoch,
        'model_name': name,
        'freeze_mode': 'freeze_to_layer1',
        'trainable_params': trainable_params,
        'total_params': total_params,
        'seed':SEED
    }


In [ ]:
# ========== Fold 2 학습 (Inception-C부터 Fine-tuning) ==========

all_fold_results = []

fold_info = fold_splits[0]  # Fold 2
fold_idx = fold_info['fold']
train_drivers = fold_info['train_drivers']
val_drivers = fold_info['val_drivers']


print(f"==== {name} ====")


fold_result = train_fold(
    fold_idx, 
    train_drivers, 
    val_drivers,
)

all_fold_results.append(fold_result)

# 메모리 정리
torch.cuda.empty_cache()

# 결과 출력
print(f"\n{'='*70}")
print("📊 최종 결과")
print(f"{'='*70}")
print(f"  Fold: {fold_result['fold']}")
print(f"  최저 Val Loss: {fold_result['best_val_loss']:.4f}")
print(f"  해당 Val Acc: {fold_result['best_val_acc']:.2f}%")
print(f"  학습 완료 에폭: {fold_result['stopped_epoch']}")
print(f"  Freeze 모드: {fold_result['freeze_mode']}")
print(f"  학습 파라미터: {fold_result['trainable_params']:,} / {fold_result['total_params']:,}")
print(f"{'='*70}")

In [ ]:
# ========== 학습 곡선 시각화 (4개 그래프) ==========

fig, axes = plt.subplots(2, 2, figsize=(16, 12))

for result in all_fold_results:
    fold_idx = result['fold']
    history = result['history']
    stopped_epoch = result['stopped_epoch']
    best_val_loss = result['best_val_loss']
    
    # 1. Loss 그래프
    ax1 = axes[0, 0]
    epochs = range(1, len(history['train_loss']) + 1)
    ax1.plot(epochs, history['train_loss'], label='Train Loss', 
             marker='o', linewidth=2, alpha=0.8, color='#1f77b4')
    ax1.plot(epochs, history['val_loss'], label='Val Loss', 
             marker='s', linewidth=2, alpha=0.8, color='#ff7f0e')
    
    # 최저 Val Loss 지점 표시
    best_epoch = np.argmin(history['val_loss']) + 1
    ax1.scatter(best_epoch, best_val_loss, color='red', s=250, zorder=5, 
                marker='*', edgecolors='black', linewidths=2,
                label=f'Best (Epoch {best_epoch})')
    
    # Early Stopping 지점 표시
    if stopped_epoch < num_epochs:
        ax1.axvline(stopped_epoch, color='red', linestyle='--', 
                   linewidth=2, alpha=0.5, label=f'Early Stop (E{stopped_epoch})')
    
    ax1.set_title(f'Fold {fold_idx} - Loss (Multiclass Log Loss)', 
                 fontsize=14, fontweight='bold')
    ax1.set_xlabel('Epoch', fontsize=12)
    ax1.set_ylabel('Loss', fontsize=12)
    ax1.legend(loc='best', fontsize=10)
    ax1.grid(True, alpha=0.3)
    
    # 2. Accuracy 그래프
    ax2 = axes[0, 1]
    ax2.plot(epochs, history['train_acc'], label='Train Acc', 
             marker='o', linewidth=2, alpha=0.8, color='#2ca02c')
    ax2.plot(epochs, history['val_acc'], label='Val Acc', 
             marker='s', linewidth=2, alpha=0.8, color='#d62728')
    
    # 최고 Val Acc 지점 표시
    best_acc_epoch = np.argmax(history['val_acc']) + 1
    best_val_acc = max(history['val_acc'])
    ax2.scatter(best_acc_epoch, best_val_acc, color='green', s=250, zorder=5,
                marker='*', edgecolors='black', linewidths=2,
                label=f'Best (Epoch {best_acc_epoch})')
    
    if stopped_epoch < num_epochs:
        ax2.axvline(stopped_epoch, color='red', linestyle='--', 
                   linewidth=2, alpha=0.5, label=f'Early Stop (E{stopped_epoch})')
    
    ax2.set_title(f'Fold {fold_idx} - Accuracy', fontsize=14, fontweight='bold')
    ax2.set_xlabel('Epoch', fontsize=12)
    ax2.set_ylabel('Accuracy (%)', fontsize=12)
    ax2.legend(loc='best', fontsize=10)
    ax2.grid(True, alpha=0.3)
    
    # 3. Learning Rate 그래프
    ax3 = axes[1, 0]
    ax3.plot(epochs, history['learning_rates'], marker='o', linewidth=2, 
             color='purple', alpha=0.8, label='Learning Rate')
    ax3.set_title(f'Fold {fold_idx} - Learning Rate Schedule', 
                 fontsize=14, fontweight='bold')
    ax3.set_xlabel('Epoch', fontsize=12)
    ax3.set_ylabel('Learning Rate', fontsize=12)
    ax3.set_yscale('log')
    ax3.legend(loc='best', fontsize=10)
    ax3.grid(True, alpha=0.3, which='both')
    
    if stopped_epoch < num_epochs:
        ax3.axvline(stopped_epoch, color='red', linestyle='--', 
                   linewidth=2, alpha=0.5)
    
    # 4. Train vs Val 비교 (Loss & Acc)
    ax4 = axes[1, 1]
    
    # Loss 차이
    loss_diff = np.array(history['train_loss']) - np.array(history['val_loss'])
    ax4_twin = ax4.twinx()
    
    ax4.plot(epochs, loss_diff, marker='o', linewidth=2, 
            color='#e377c2', alpha=0.7, label='Loss Diff (Train - Val)')
    ax4.axhline(0, color='gray', linestyle='--', linewidth=1)
    ax4.set_xlabel('Epoch', fontsize=12)
    ax4.set_ylabel('Loss Difference', fontsize=12, color='#e377c2')
    ax4.tick_params(axis='y', labelcolor='#e377c2')
    
    # Accuracy 차이
    acc_diff = np.array(history['train_acc']) - np.array(history['val_acc'])
    ax4_twin.plot(epochs, acc_diff, marker='s', linewidth=2,
                 color='#bcbd22', alpha=0.7, label='Acc Diff (Train - Val)')
    ax4_twin.set_ylabel('Accuracy Difference (%)', fontsize=12, color='#bcbd22')
    ax4_twin.tick_params(axis='y', labelcolor='#bcbd22')
    
    ax4.set_title(f'Fold {fold_idx} - Overfitting 모니터링', 
                 fontsize=14, fontweight='bold')
    ax4.grid(True, alpha=0.3)
    
    # Legend 통합
    lines1, labels1 = ax4.get_legend_handles_labels()
    lines2, labels2 = ax4_twin.get_legend_handles_labels()
    ax4.legend(lines1 + lines2, labels1 + labels2, loc='best', fontsize=9)
    
    if stopped_epoch < num_epochs:
        ax4.axvline(stopped_epoch, color='red', linestyle='--', 
                   linewidth=2, alpha=0.5)

plt.tight_layout()
os.makedirs(f'./plots/{name}', exist_ok=True)
plt.savefig(f'./plots/{name}/{name}_losscurve_detailed.png', 
           dpi=300, bbox_inches='tight')
plt.show()

# ========== 통계 출력 ==========
print("\n" + "="*70)
print(f"📈 {name} 학습 통계 상세")
print("="*70)
print(f"초기 Train Loss: {history['train_loss'][0]:.4f}")
print(f"최종 Train Loss: {history['train_loss'][-1]:.4f}")
print(f"최저 Train Loss: {min(history['train_loss']):.4f} (Epoch {np.argmin(history['train_loss'])+1})")
print(f"\n초기 Val Loss: {history['val_loss'][0]:.4f}")
print(f"최종 Val Loss: {history['val_loss'][-1]:.4f}")
print(f"최저 Val Loss: {min(history['val_loss']):.4f} (Epoch {np.argmin(history['val_loss'])+1})")
print(f"\n초기 Train Acc: {history['train_acc'][0]:.2f}%")
print(f"최종 Train Acc: {history['train_acc'][-1]:.2f}%")
print(f"최고 Train Acc: {max(history['train_acc']):.2f}% (Epoch {np.argmax(history['train_acc'])+1})")
print(f"\n초기 Val Acc: {history['val_acc'][0]:.2f}%")
print(f"최종 Val Acc: {history['val_acc'][-1]:.2f}%")
print(f"최고 Val Acc: {max(history['val_acc']):.2f}% (Epoch {np.argmax(history['val_acc'])+1})")
print(f"\n초기 LR: {history['learning_rates'][0]:.6f}")
print(f"최종 LR: {history['learning_rates'][-1]:.6f}")
print(f"LR 변경 횟수: {len(set(history['learning_rates'])) - 1}회")
print("="*70)

# ========== Overfitting 분석 ==========
final_loss_gap = history['train_loss'][-1] - history['val_loss'][-1]
final_acc_gap = history['train_acc'][-1] - history['val_acc'][-1]

print("\n" + "="*70)
print("🔍 Overfitting 분석")
print("="*70)
print(f"최종 Loss 차이 (Train - Val): {final_loss_gap:+.4f}")
print(f"최종 Acc 차이 (Train - Val): {final_acc_gap:+.2f}%")

if final_acc_gap > 10:
    print("⚠️ 경고: 심각한 Overfitting 감지! (Acc 차이 > 10%)")
elif final_acc_gap > 5:
    print("⚠️ 주의: 약간의 Overfitting 감지 (Acc 차이 > 5%)")
else:
    print("✅ 양호: Overfitting이 잘 제어되고 있습니다.")

print("="*70)

# Submission

In [ ]:
print("\n" + "=" * 70)
print("🔮 테스트 데이터 예측 시작")
print("=" * 70)

driver_df = pd.read_csv(driver_csv_path)

# 1. 테스트 데이터셋 생성
test_dataset = DriverDataset(
    test_dir, driver_df, [], 
    transform=transform_eval, is_test=True
)

test_loader = DataLoader(
    test_dataset,
    batch_size=batch_size,
    shuffle=False,
    num_workers=num_workers,
    pin_memory=True
)

print(f"테스트 샘플: {len(test_dataset)}개")

In [ ]:
# ========== Submission 파일 생성 (학습된 모델 사용) ==========



# 2. 모델 생성
model = ResNetDWithHead(
    num_classes=num_classes,
    embed_dim=EMBED_DIM,
    dropout=DROPOUT,
).to(device)

# 3. 저장된 가중치 로드
best_model_path = f'models/{name}.pth'
print(f"\n📁 모델 로드 중: {best_model_path}...")

try:
    checkpoint = torch.load(best_model_path, map_location=device)
    model.load_state_dict(checkpoint['model_state_dict'])
    
    # 로드된 모델 정보 출력
    print(f"✅ 모델 로드 완료!")
    print(f"   - Fold: {checkpoint.get('fold', 'N/A')}")
    print(f"   - Epoch: {checkpoint.get('epoch', 'N/A')+1}")
    print(f"   - Val Loss: {checkpoint.get('val_loss', 'N/A'):.4f}")
    print(f"   - Val logLoss: {checkpoint.get('val_logloss', 'N/A'):.4f}")
    print(f"   - Val Acc: {checkpoint.get('val_acc', 'N/A'):.2f}%")
    print(f"   - Val Macro-F1: {checkpoint.get('val_macro_f1', 'N/A'):.4f}")
    print(f"   - Val LogLoss: {checkpoint.get('val_logloss', 'N/A'):.4f}")
    
except Exception as e:
    print(f"🚨 모델 로드 실패: {e}")
    raise

model.eval()

# 4. 예측 실행
predictions = []
img_names = []

print("\n🔮 예측 진행 중...")
with torch.no_grad():
    for images, filenames in tqdm(test_loader, desc='예측'):
        images = images.to(device, non_blocking=True)
        outputs = model(images)
        
        # 5. 확률 (Softmax) 계산
        probs = torch.softmax(outputs, dim=1)
        predictions.append(probs.cpu().numpy())
        img_names.extend(filenames)

final_predictions = np.vstack(predictions)
print(f"\n✅ 예측 완료: {final_predictions.shape}")

# 6. Submission 파일 생성
class_cols = [f'c{i}' for i in range(num_classes)]

submission_data = {'img': img_names}
for i, col in enumerate(class_cols):
    submission_data[col] = final_predictions[:, i]

submission = pd.DataFrame(submission_data)

# 7. 파일 저장
os.makedirs('./submissions', exist_ok=True)
submission.to_csv(f"./submissions/{name}_submission.csv", index=False)

print("\n" + "=" * 70)
print(f"✅ Submission 파일 생성 완료!")
print("=" * 70)
print(f"📁 저장 경로: ./submissions/{name}_submission.csv")
print(f"📊 총 예측 이미지: {len(submission)}개")
print(f"📋 클래스별 평균 확률:")
for col in class_cols:
    print(f"   {col}: {submission[col].mean():.4f}")
print("=" * 70)

print("\n📋 Submission 샘플 (처음 5개):")
print(submission.head())

print("\n📋 Submission 샘플 (마지막 5개):")
print(submission.tail())

# 8. 확률 분포 검증 (합이 1인지 확인)
prob_sums = submission[class_cols].sum(axis=1)
print(f"\n✅ 확률 합 검증: min={prob_sums.min():.6f}, max={prob_sums.max():.6f}, mean={prob_sums.mean():.6f}")
if not np.allclose(prob_sums, 1.0, atol=1e-5):
    print("⚠️ 경고: 일부 샘플의 확률 합이 1이 아닙니다!")
else:
    print("✅ 모든 샘플의 확률 합이 1입니다.")